اول امپورت کتاب خونه هایی که لازم داریم

In [ ]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

فایل های سی اس وی رو میخونیم و توی متقیر های جدا ذخیره میکنیم

In [ ]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

نشون دادن 

In [ ]:
movies_df.head()

سال و تایتل رو حذف میکنیم و میبریمش توی یه تایتل جدا

In [ ]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

دوباره نمایشش بده

In [ ]:
movies_df.head()

مستقیم ژانر رو جدا میکنیم

In [ ]:
movies_df = movies_df.drop('genres', 1)

دوباره نشونش میدیم 

In [ ]:
movies_df.head()

حالا متقیر دوم رو نشون میدیم

In [ ]:
ratings_df.head()

تایم استمپ رو از متقیر ریتینگ جدا میکنیم

In [ ]:
ratings_df = ratings_df.drop('timestamp', 1)

و دوباره نشونش میدیم که ببینیم چه شکلی شده

In [ ]:
ratings_df.head()

به صورت دل بخواهی یه نفر زو که درست میکنیم و از نضر خودمون به فیلم ها امتیاز میدیم 
و بعد یوزر امپوت رو توی یه متقیر به اسم ایمپوت موری ذخیره میکنم

In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

و میایم مو وی ایدی رو هم به جدول اضافه میکنیم
مو وی ایدی رو از توی تایتل به متقیر انپوت ایدی ازافه میکنیم
ایمپوت مو وری و امپوت ای دی رو با هم ترکیب میکنیم و توی متقیر ایمپوت مو وری ویریزیم
و بعد سال رو از توی ایمو.ت مووی حذف میکنیم
و چاپش کن

In [ ]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('year', 1)
inputMovies

حالا من میخوام دنبال یوزر هایی بگردم که این فیلم ها رو دیدن و شبیه یوزر من هستن

ریتینگ دیتا فریم در حالتی که 
مووی ایدی که بهش امتیاز داده توی امپوت مووی هایی باشه که توی مووی باشه
و توی یوزر ست ذخیره میکنم 
و چاپش میکنم

In [ ]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

یه متقیر به اسم یوزر سابت گروپ درست میکنیم 
یوزر ساذف رو گروپ بای میکنیم 
توی یوزر ایدی 
و توی متقیر ذخیره میکنیم

In [ ]:
userSubsetGroup = userSubset.groupby(['userId'])

حالا شماره یوزر رو میدیم و دیتا هاش رو نشون میده

In [ ]:
userSubsetGroup.get_group(1130)

کل یوزر ساپت ست گروپ رو سورت کن 
و تعداد گروه هایی که هست رو به من بده 
و بریزش توی متقیر به اسم یوزر ساپت گروپ

In [ ]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

و سه ردیف اول یوزر سابت گروپ رو بهم بده

In [ ]:
userSubsetGroup[0:3]

صد نفری که بیشترین تعداد نفراتی که فیلم هایی مثل یوزر من رو داشتن

و با این کار برنامه خیلی سریع تر کار میکنه

In [ ]:
userSubsetGroup = userSubsetGroup[0:100]

pearsonCorrelationDictخالیه
و بعد کل الگوریتم رو اجرا کن 
الگوریتم اینجوری کار میکنه که
یه دیکشنری خالی هست
هر بار برای هر ادمی در گروهش یوزر ساپت گروپ انتخاب میکنم
و بعد گروه ها رو بر اساس چیز هایی که هست رو مرتب کرده و سورت میکنیم
تعداد فیلم های مشترک رو تایید کن
تعداد ستاره هایی که برای فیلم های مشترک هست
گروه رو مشخص کرده
و در اخر فرمول رو پیاده سازی میکنیم 

حالا با یه ایف اگر sxx صفر نبود و syy هم صفر نبود 
    جواب پیرسون من رو توی این فرمول حل کن
در قیر این صورت 
    پیرسون من رو صفر قرار بده

In [ ]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


نمایش دادن ماتریس

In [ ]:
pearsonCorrelationDict.items()

ماتریسی که بالا داشتیم رو به صورت جدول نشون میدیم 
و مرتب ترش میکنیم

In [ ]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

حالا پیرسون دی اف رو سورت کن 
similarityIndexبر اساس ستون
و نشونش بده

In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

تاو یوزر ریتین ها رو مرج کن روی دیا ریتینگ فریمی که داشتم
و نشونش بده

In [ ]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

سیمی لارتی ریتینگز رو ضربه ایندکس کن
و نشونش بده
و کارش اینه که امتیازی که کاربر به اون فیلمی که من دیدم چقدره و توی شباهتش ما من حصاب کن

In [ ]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

topUsersRating رو بر اساس مووی ایدی جدا کن
و ببین که به فیلم یک به چه مجموعی میرسه
اسم ستون ها رو هم ست میکنیم
و چاپش میکنیم

In [ ]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

حالا یه دیتا فریم جدید درست میکنیم 


و چاپش کن

In [ ]:
recommendation_df = pd.DataFrame()
#و این فرمول رو توش میذاریم
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

حالا برای اینکه ببینیم چی باید بهش ویشنهاد بده 
دیتا رو بر اساس ولیویی که بهش دادیم سورت کنیم
و ده تای اول رو چاپ میکنیم تا ویشنهاد هارو ببینیم

In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

برای اینکه اسم فیلم ها رو هم بدم
موویز دیتا فریم رو چک کن روی ستون ها 

In [ ]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]